In [1]:
import os
import pickle
import numpy as np

import torch
import torch.nn.functional
from torchsummary import summary

from pyeer.eer_info import get_eer_stats
from pyeer.report import generate_eer_report, export_error_rates
from pyeer.plot import plot_eer_stats

from config import models_folder, output_data_folder, EER_folder

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [3]:
model_folder = os.path.join(models_folder, "verification_classifier", "good_models", "2020-04-06_16-34-21")

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
torch.cuda.is_available()

cuda


True

In [5]:
# Verification Binary classifier
model = torch.jit.load(os.path.join(model_folder, "mobile_model.pt"))
model.to(device)

ScriptModule(
  (encoder): ScriptModule(
    (encoder): ScriptModule(
      (features): ScriptModule(
        (conv0): ScriptModule()
        (norm0): ScriptModule()
        (relu0): ScriptModule()
        (pool0): ScriptModule()
        (denseblock1): ScriptModule(
          (denselayer1): ScriptModule(
            (norm1): ScriptModule()
            (relu1): ScriptModule()
            (conv1): ScriptModule()
            (norm2): ScriptModule()
            (relu2): ScriptModule()
            (conv2): ScriptModule()
          )
          (denselayer2): ScriptModule(
            (norm1): ScriptModule()
            (relu1): ScriptModule()
            (conv1): ScriptModule()
            (norm2): ScriptModule()
            (relu2): ScriptModule()
            (conv2): ScriptModule()
          )
          (denselayer3): ScriptModule(
            (norm1): ScriptModule()
            (relu1): ScriptModule()
            (conv1): ScriptModule()
            (norm2): ScriptModule()
            (rel

In [6]:
# Validation data
validation_set_file = os.path.join(output_data_folder, "validation_sets", "verification_validation_set.pickle")
with open(validation_set_file, 'rb') as f:
    validation_data = pickle.load(f)

In [7]:
class_probs_all = []
outputs_all = []
labels_all = []
geniune_scores_all = []
impostor_scores_all = []

for data in validation_data:
    # prep inputs
    input_imgs, labels = data
    inputs = [img.to(device) for img in input_imgs]
    # predict
    with torch.no_grad():
        model.eval()   # eval mode
        outputs = model(inputs)
        class_probs = torch.nn.functional.softmax(outputs, dim=1)
    
    # save model outputs
    # Convert to numpy
    class_probs = class_probs.cpu().detach().numpy()
    outputs = outputs.cpu().detach().numpy()
    labels = labels.cpu().detach().numpy()
    # append arrays
    class_probs_all.append(class_probs)
    outputs_all.append(outputs)
    labels_all.append(labels)
    # genuine/impostor scores
    genuine_idx = np.argwhere(labels == 1)
    imposter_idx = np.argwhere(labels == 0)
    # append values
    geniune_scores_all.extend(class_probs[genuine_idx, 1].flatten().tolist())
    impostor_scores_all.extend(class_probs[imposter_idx, 1].flatten().tolist())
    

In [8]:
# Scores folder
scores_folder = os.path.join(EER_folder, os.path.basename(model_folder))
if not os.path.exists(scores_folder): os.makedirs(scores_folder)

# Save scores
scores_save_config = [
    (class_probs_all, "class_probs_all.pickle"),
    (outputs_all, "outputs_all.pickle"),
    (labels_all, "labels_all.pickle"),
    (geniune_scores_all, "geniune_scores_all.pickle"),
    (impostor_scores_all, "impostor_scores_all.pickle"),
]
for list_, filename in scores_save_config:
    file = os.path.join(scores_folder, filename)
    with open(file, 'wb') as handle:
        pickle.dump(list_, handle, protocol=pickle.HIGHEST_PROTOCOL)
    # Print count stats
    print("{}, Count: {}".format(filename, len(list_)))

class_probs_all.pickle, Count: 12
outputs_all.pickle, Count: 12
labels_all.pickle, Count: 12
geniune_scores_all.pickle, Count: 1920
impostor_scores_all.pickle, Count: 1920


In [9]:
# EER Stats
# Calculating stats for classifier A
eer_stats = get_eer_stats(geniune_scores_all, impostor_scores_all)
# Generating report
generate_eer_report([eer_stats], ["model_a"], os.path.join(scores_folder, 'pyeer_report.csv'))
generate_eer_report([eer_stats], ["model_a"], os.path.join(scores_folder, 'pyeer_report.html'))
# Exporting error rates (Exporting FMR and FNMR to a CSV file)
# This is the DET curve, the ROC curve is a plot of FMR against 1 - FNMR
export_error_rates(eer_stats.fmr, eer_stats.fnmr, os.path.join(scores_folder, 'DET.csv'))
# Plotting
plot_eer_stats([eer_stats], ['model_a'], save_path=scores_folder)